# Homework and bake-off: Word similarity

In [292]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Dataset readers](#Dataset-readers)
1. [Dataset comparisons](#Dataset-comparisons)
  1. [Vocab overlap](#Vocab-overlap)
  1. [Pair overlap and score correlations](#Pair-overlap-and-score-correlations)
1. [Evaluation](#Evaluation)
  1. [Dataset evaluation](#Dataset-evaluation)
  1. [Dataset error analysis](#Dataset-error-analysis)
  1. [Full evaluation](#Full-evaluation)
1. [Homework questions](#Homework-questions)
  1. [PPMI as a baseline [0.5 points]](#PPMI-as-a-baseline-[0.5-points])
  1. [Gigaword with LSA at different dimensions [0.5 points]](#Gigaword-with-LSA-at-different-dimensions-[0.5-points])
  1. [Gigaword with GloVe [0.5 points]](#Gigaword-with-GloVe-[0.5-points])
  1. [Dice coefficient [0.5 points]](#Dice-coefficient-[0.5-points])
  1. [t-test reweighting [2 points]](#t-test-reweighting-[2-points])
  1. [Enriching a VSM with subword information [2 points]](#Enriching-a-VSM-with-subword-information-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

Word similarity datasets have long been used to evaluate distributed representations. This notebook provides basic code for conducting such analyses with a number of datasets:

| Dataset | Pairs | Task-type | Current best Spearman $\rho$ | Best $\rho$ paper |   |
|---------|-------|-----------|------------------------------|-------------------|---|
| [WordSim-353](http://www.gabrilovich.com/resources/data/wordsim353/) | 353 | Relatedness | 82.8 | [Speer et al. 2017](https://arxiv.org/abs/1612.03975) |
| [MTurk-771](http://www2.mta.ac.il/~gideon/mturk771.html) | 771 | Relatedness | 81.0 | [Speer et al. 2017](https://arxiv.org/abs/1612.03975) |
| [The MEN Test Collection](https://staff.fnwi.uva.nl/e.bruni/MEN) | 3,000 | Relatedness | 86.6 | [Speer et al. 2017](https://arxiv.org/abs/1612.03975)  | 
| [SimVerb-3500-dev](https://www.aclweb.org/anthology/D16-1235/) | 500 | Similarity | 61.1 | [Mrki&scaron;&cacute; et al. 2016](https://arxiv.org/pdf/1603.00892.pdf) |
| [SimVerb-3500-test](https://www.aclweb.org/anthology/D16-1235/) | 3,000 | Similarity | 62.4 | [Mrki&scaron;&cacute; et al. 2016](https://arxiv.org/pdf/1603.00892.pdf) |

Each of the similarity datasets contains word pairs with an associated human-annotated similarity score. (We convert these to distances to align intuitively with our distance measure functions.) The evaluation code measures the distance between the word pairs in your chosen VSM (which should be a `pd.DataFrame`).

The evaluation metric for each dataset is the [Spearman correlation coefficient $\rho$](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) between the annotated scores and your distances, as is standard in the literature. We also macro-average these correlations across the datasets for an overall summary. (In using the macro-average, we are saying that we care about all the datasets equally, even though they vary in size.)

This homework ([questions at the bottom of this notebook](#Homework-questions)) asks you to write code that uses the count matrices in `data/vsmdata` to create and evaluate some baseline models as well as an original model $M$ that you design. This accounts for 9 of the 10 points for this assignment.

For the associated bake-off, we will distribute two new word similarity or relatedness datasets and associated reader code, and you will evaluate $M$ (no additional training or tuning allowed!) on those new datasets. Systems that enter will receive the additional homework point, and systems that achieve the top score will receive an additional 0.5 points.

## Set-up

In [293]:
from collections import defaultdict
import csv
import itertools
import numpy as np
import os
import pandas as pd
from scipy.stats import spearmanr
import vsm
from IPython.display import display

In [294]:
VSM_HOME = os.path.join('data', 'vsmdata')

WORDSIM_HOME = os.path.join('data', 'wordsim')

In [295]:
os.listdir(VSM_HOME)

['._gigawordnyt-advmod-matrix.csv.gz',
 'giga-wordcounts.pickle',
 'gigawordnyt-advmod-matrix.csv.gz',
 'giga_window20-flat.csv.gz',
 'giga_window5-scaled.csv.gz',
 'imdb-wordcounts.pickle',
 'imdb_window20-flat.csv.gz',
 'imdb_window5-scaled.csv.gz']

In [8]:
os.listdir(WORDSIM_HOME)

['._MEN',
 '._SimVerb-3500',
 '._wordsim353',
 'MEN',
 'MTURK-771.csv',
 'SimVerb-3500',
 'wordsim353']

## Dataset readers

In [296]:
def wordsim_dataset_reader(src_filename, header=False, delimiter=',', score_col_index=2):
    """
    Basic reader that works for all similarity datasets. They are
    all tabular-style releases where the first two columns give the
    word and a later column (`score_col_index`) gives the score.

    Parameters
    ----------
    src_filename : str
        Full path to the source file.

    header : bool
        Whether `src_filename` has a header.

    delimiter : str
        Field delimiter in `src_filename`.

    score_col_index : int
        Column containing the similarity scores Default: 2

    Yields
    ------
    (str, str, float)
       (w1, w2, score) where `score` is the negative of the similarity
       score in the file so that we are intuitively aligned with our
       distance-based code. To align with our VSMs, all the words are
       downcased.

    """
    with open(src_filename) as f:
        reader = csv.reader(f, delimiter=delimiter)
        if header:
            next(reader)
        for row in reader:
            w1 = row[0].strip().lower()
            w2 = row[1].strip().lower()
            score = row[score_col_index]
            # Negative of scores to align intuitively with distance functions:
            score = -float(score)
            yield (w1, w2, score)

def wordsim353_reader():
    """WordSim-353: http://www.gabrilovich.com/resources/data/wordsim353/"""
    src_filename = os.path.join(WORDSIM_HOME, 'wordsim353', 'combined.csv')
    return wordsim_dataset_reader(src_filename, header=True)

def mturk771_reader():
    """MTURK-771: http://www2.mta.ac.il/~gideon/mturk771.html"""
    src_filename = os.path.join(WORDSIM_HOME, 'MTURK-771.csv')
    return wordsim_dataset_reader(src_filename, header=False)

def simverb3500dev_reader():
    """SimVerb-3500: https://www.aclweb.org/anthology/D16-1235/"""
    src_filename = os.path.join(WORDSIM_HOME, 'SimVerb-3500', 'SimVerb-500-dev.txt')
    return wordsim_dataset_reader(src_filename, delimiter="\t", header=False, score_col_index=3)

def simverb3500test_reader():
    """SimVerb-3500: https://www.aclweb.org/anthology/D16-1235/"""
    src_filename = os.path.join(WORDSIM_HOME, 'SimVerb-3500', 'SimVerb-3000-test.txt')
    return wordsim_dataset_reader(src_filename, delimiter="\t", header=False, score_col_index=3)

def men_reader():
    """MEN: https://staff.fnwi.uva.nl/e.bruni/MEN"""
    src_filename = os.path.join(WORDSIM_HOME, 'MEN', 'MEN_dataset_natural_form_full')
    return wordsim_dataset_reader(src_filename, header=False, delimiter=' ')

In [317]:
men = men_reader()
sim = simverb3500test_reader()
simt = simverb3500dev_reader()
turk = mturk771_reader()
word = wordsim353_reader()

In [318]:
df = pd.DataFrame(columns=['w1','w2','sim'])
def extractor(generator):
    lis = []
    for i in generator:
        lis.append(list(i))
    temp_df = pd.DataFrame(lis,columns=['w1','w2','sim'])
    return temp_df

In [319]:
men2 = extractor(men)
df = pd.concat([df,men2])
sim2 = extractor(sim)
df = pd.concat([df, sim2])
simt2 = extractor(simt)
df = pd.concat([df, simt2])
turk2 = extractor(turk)
df = pd.concat([df, turk2])
word = extractor(word)
df = pd.concat([df, word])

In [320]:
df.shape

(7624, 3)

In [231]:
df = pd.DataFrame(columns=['w1','w2','sim'])
lis = []
for i in men or sim or simt or turk or word:
#     temp_df = pd.DataFrame(list(i))
    lis.append(list(i))
    print(i)
    df = pd.concat([df,temp_df])

('sun', 'sunlight', -50.0)
('automobile', 'car', -50.0)
('river', 'water', -49.0)
('stairs', 'staircase', -49.0)
('morning', 'sunrise', -49.0)
('rain', 'storm', -49.0)
('cat', 'kittens', -49.0)
('dance', 'dancers', -49.0)
('camera', 'photography', -49.0)
('cat', 'feline', -48.0)
('sunny', 'sunshine', -48.0)
('pregnancy', 'pregnant', -48.0)
('beach', 'sand', -48.0)
('bakery', 'bread', -48.0)
('flowers', 'garden', -48.0)
('grass', 'lawn', -48.0)
('copper', 'metal', -48.0)
('photos', 'photography', -47.0)
('cemetery', 'graveyard', -47.0)
('gravestone', 'graveyard', -47.0)
('sun', 'sunshine', -47.0)
('black', 'dark', -47.0)
('cathedral', 'church', -47.0)
('frozen', 'ice', -47.0)
('station', 'subway', -47.0)
('children', 'kids', -46.0)
('aquarium', 'fish', -46.0)
('light', 'lighting', -46.0)
('fungi', 'mushrooms', -46.0)
('frost', 'snow', -46.0)
('burn', 'flame', -46.0)
('ocean', 'sea', -46.0)
('candy', 'chocolate', -46.0)
('car', 'vehicle', -46.0)
('concert', 'music', -46.0)
('photos', 'pi

('stairs', 'wall', -28.0)
('chapel', 'ruins', -28.0)
('beef', 'tomato', -28.0)
('doodle', 'scratch', -28.0)
('daffodils', 'purple', -28.0)
('lake', 'scenery', -28.0)
('roof', 'staircase', -28.0)
('boys', 'living', -28.0)
('chain', 'store', -28.0)
('coast', 'hill', -28.0)
('mushrooms', 'pepper', -28.0)
('porch', 'window', -28.0)
('hot', 'water', -28.0)
('socks', 'white', -28.0)
('arch', 'bridge', -28.0)
('dessert', 'raspberry', -28.0)
('brick', 'ceiling', -28.0)
('scenery', 'sunshine', -28.0)
('city', 'square', -28.0)
('stand', 'walk', -28.0)
('evening', 'fun', -28.0)
('happy', 'sexy', -28.0)
('burger', 'mac', -28.0)
('painted', 'sculpture', -28.0)
('misty', 'morning', -28.0)
('frost', 'rain', -28.0)
('color', 'shade', -28.0)
('bacon', 'potatoes', -28.0)
('patio', 'stairs', -28.0)
('hot', 'washing', -27.0)
('button', 'stickers', -27.0)
('cute', 'happy', -27.0)
('happy', 'tears', -27.0)
('pepper', 'soup', -27.0)
('alley', 'racing', -27.0)
('smile', 'tears', -27.0)
('train', 'underground'

('abstract', 'candles', -13.0)
('church', 'soldiers', -13.0)
('boxer', 'reflection', -13.0)
('crystal', 'silhouette', -13.0)
('feel', 'guy', -13.0)
('pattern', 'pier', -13.0)
('downtown', 'field', -13.0)
('beer', 'wild', -13.0)
('hanging', 'happy', -13.0)
('rope', 'shore', -13.0)
('airplane', 'camera', -13.0)
('button', 'flowers', -13.0)
('frozen', 'shop', -13.0)
('curled', 'licking', -13.0)
('sunny', 'zoo', -13.0)
('people', 'stop', -13.0)
('bucket', 'insects', -13.0)
('fence', 'friends', -13.0)
('can', 'sketch', -13.0)
('frozen', 'pebbles', -13.0)
('ceramic', 'living', -13.0)
('day', 'horse', -13.0)
('born', 'boxer', -13.0)
('iris', 'necklace', -13.0)
('lighting', 'person', -13.0)
('boot', 'cliff', -13.0)
('landscape', 'umbrella', -13.0)
('man', 'transformers', -13.0)
('dark', 'sand', -13.0)
('ruins', 'swimsuit', -13.0)
('abstract', 'rose', -13.0)
('guy', 'gymnastics', -12.0)
('grass', 'ruins', -12.0)
('lighting', 'stone', -12.0)
('porch', 'scooter', -12.0)
('colour', 'wild', -12.0)


In [232]:
l = pd.DataFrame(lis)
df =pd.concat([df,l])

In [321]:
df.sort_values(by='w1')

,w1,w2,sim
1025,abandoned,ruins,-32.00
1972,abandoned,railroad,-18.00
2643,abandoned,frost,-9.00
1398,abandoned,soldiers,-27.00
2667,abandoned,button,-9.00
...,...,...,...
1736,yield,give,-6.31
2682,yield,aid,-2.99
1579,yield,wait,-5.64
403,yield,submit,-6.64


In [326]:
arr = df.

In [332]:
arr.w1

<xarray.DataArray 'w1' (index: 7624)>
array(['sun', 'automobile', 'river', ..., 'disaster', 'governor',
       'architecture'], dtype=object)
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 7 8 ... 345 346 347 348 349 350 351 352

In [324]:
df.dot()

ImportError: Missing optional dependency 'xarray'.  Use pip or conda to install xarray.

In [165]:
wordsim = pd.read_csv(os.path.join(WORDSIM_HOME, 'wordsim353', 'combined.csv'))
mturk = pd.read_csv(os.path.join(WORDSIM_HOME, 'MTURK-771.csv'))
simverbd = pd.read_csv(os.path.join(WORDSIM_HOME, 'SimVerb-3500', 'SimVerb-500-dev.txt'), sep = '\t')
simverbd = simverbd[['hurt','offend','6.81']]
simvebt = pd.read_csv(os.path.join(WORDSIM_HOME, 'SimVerb-3500', 'SimVerb-3000-test.txt'), sep = '\t')
simvebt = simvebt[['walk', 'trail', '4.81']]
men = pd.read_csv(os.path.join(WORDSIM_HOME, 'MEN', 'MEN_dataset_natural_form_full'),sep = ' ')

In [166]:
for name in READERS:
    test = wordsim_dataset_reader(name)
    print(test)

<generator object wordsim_dataset_reader at 0x000001EC6E7F34C8>
<generator object wordsim_dataset_reader at 0x000001EC6E7F3548>
<generator object wordsim_dataset_reader at 0x000001EC6E7F34C8>
<generator object wordsim_dataset_reader at 0x000001EC6E7F3548>
<generator object wordsim_dataset_reader at 0x000001EC6E7F34C8>


In [167]:
get_reader_name(READERS)

NameError: name 'get_reader_name' is not defined

In [164]:
print(test)

<generator object wordsim_dataset_reader at 0x0000023B2982B4C8>


In [158]:
print(t)

<built-in method close of generator object at 0x0000023B2982BB48>


In [140]:
dir(test)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

This collection of readers will be useful for flexible evaluations:

In [10]:
READERS = (wordsim353_reader, mturk771_reader, simverb3500dev_reader,
           simverb3500test_reader, men_reader)

In [35]:
set1 = ['ass','2','fuck','ass','ass','ass']
set2 = ['dick','2','dick','ass','ass','ass','ass']

In [36]:
sets = set(set1)
sets.intersection()

{'2', 'ass', 'fuck'}

In [38]:
sets.intersection(set2)

{'2', 'ass'}

In [39]:
sets.union(set2)

{'2', 'ass', 'dick', 'fuck'}

In [31]:
dir(sets)

['__and__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iand__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__isub__',
 '__iter__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__rsub__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__xor__',
 'add',
 'clear',
 'copy',
 'difference',
 'difference_update',
 'discard',
 'intersection',
 'intersection_update',
 'isdisjoint',
 'issubset',
 'issuperset',
 'pop',
 'remove',
 'symmetric_difference',
 'symmetric_difference_update',
 'union',
 'update']

In [50]:
dir(itertools)

['__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_grouper',
 '_tee',
 '_tee_dataobject',
 'accumulate',
 'chain',
 'combinations',
 'combinations_with_replacement',
 'compress',
 'count',
 'cycle',
 'dropwhile',
 'filterfalse',
 'groupby',
 'islice',
 'permutations',
 'product',
 'repeat',
 'starmap',
 'takewhile',
 'tee',
 'zip_longest']

In [49]:
test1 = ["a",'b','c']
test2 = [1*20,2*20,3*20,4*20,5*20,6*20,7*20,8*20,9*20]

for r1, r2, r3 in itertools.product(test1, repeat=3):
    print(r1,r2, r3)
#     v1 = get_reader_vocab(r1)
#     v2 = get_reader_vocab(r2)

a a a
a a b
a a c
a b a
a b b
a b c
a c a
a c b
a c c
b a a
b a b
b a c
b b a
b b b
b b c
b c a
b c b
b c c
c a a
c a b
c a c
c b a
c b b
c b c
c c a
c c b
c c c


## Dataset comparisons

This section does some basic analysis of the datasets. The goal is to obtain a deeper understanding of what problem we're solving – what strengths and weaknesses the datasets have and how they relate to each other. For a full-fledged project, we would want to continue work like this and report on it in the paper, to provide context for the results.

In [21]:
def get_reader_name(reader):
    """
    Return a cleaned-up name for the dataset iterator `reader`.
    """
    return reader.__name__.replace("_reader", "")

### Vocab overlap

How many vocabulary items are shared across the datasets?

In [22]:
def get_reader_vocab(reader):
    """Return the set of words (str) in `reader`."""
    vocab = set()
    for w1, w2, _ in reader():
        vocab.add(w1)
        vocab.add(w2)
    return vocab

In [57]:
def get_reader_vocab_overlap(readers=READERS):
    """
    Get data on the vocab-level relationships between pairs of
    readers. Returns a a pd.DataFrame containing this information.
    """
    data = []
    for r1, r2 in itertools.product(readers, repeat=2):
        v1 = get_reader_vocab(r1)
        v2 = get_reader_vocab(r2)
        d = {
            'd1': get_reader_name(r1),
            'd2': get_reader_name(r2),
            'overlap': len(v1 & v2),
            'union': len(v1 | v2),
            'd1_size': len(v1),
            'd2_size': len(v2)}
        data.append(d)
    return pd.DataFrame(data)

In [265]:
dir(itertools)

['__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_grouper',
 '_tee',
 '_tee_dataobject',
 'accumulate',
 'chain',
 'combinations',
 'combinations_with_replacement',
 'compress',
 'count',
 'cycle',
 'dropwhile',
 'filterfalse',
 'groupby',
 'islice',
 'permutations',
 'product',
 'repeat',
 'starmap',
 'takewhile',
 'tee',
 'zip_longest']

In [302]:
def get_reader_text(reader):
    """Return the set of words (str) in `reader`."""
    vocab = []
    for w1, w2, sim in reader():
        vocab.append([w1,w2,sim])
#         vocab.add(w2)
#         vocab.add(sim)
    return vocab

In [303]:
get_reader_text(wordsim353_reader)

[['love', 'sex', -6.77],
 ['tiger', 'cat', -7.35],
 ['tiger', 'tiger', -10.0],
 ['book', 'paper', -7.46],
 ['computer', 'keyboard', -7.62],
 ['computer', 'internet', -7.58],
 ['plane', 'car', -5.77],
 ['train', 'car', -6.31],
 ['telephone', 'communication', -7.5],
 ['television', 'radio', -6.77],
 ['media', 'radio', -7.42],
 ['drug', 'abuse', -6.85],
 ['bread', 'butter', -6.19],
 ['cucumber', 'potato', -5.92],
 ['doctor', 'nurse', -7.0],
 ['professor', 'doctor', -6.62],
 ['student', 'professor', -6.81],
 ['smart', 'student', -4.62],
 ['smart', 'stupid', -5.81],
 ['company', 'stock', -7.08],
 ['stock', 'market', -8.08],
 ['stock', 'phone', -1.62],
 ['stock', 'cd', -1.31],
 ['stock', 'jaguar', -0.92],
 ['stock', 'egg', -1.81],
 ['fertility', 'egg', -6.69],
 ['stock', 'live', -3.73],
 ['stock', 'life', -0.92],
 ['book', 'library', -7.46],
 ['bank', 'money', -8.12],
 ['wood', 'forest', -7.73],
 ['money', 'cash', -9.15],
 ['professor', 'cucumber', -0.31],
 ['king', 'cabbage', -0.23],
 ['kin

In [312]:
def get_reader_data(readers=READERS):
    """
    Get data on the vocab-level relationships between pairs of
    readers. Returns a a pd.DataFrame containing this information.
    """
    data = []
    for r1 in itertools.product(readers,repeat=1):
        w1 = get_reader_data(r1)
        print(w1)
        break
#         w2 = get_reader_vocab(r2)
#         sim = get_reader_data(r3)
#         d = {
#             'w1': get_reader_name(r1),
#             'w2': get_reader_name(r2),
#             'sim': len(v1 & v2),
#             'union': len(v1 | v2),
#             'd1_size': len(v1),
#             'd2_size': len(v2)}
#         data.append(d)
#     return pd.DataFrame(data)

In [313]:
get_reader_data()

RecursionError: maximum recursion depth exceeded while calling a Python object

In [58]:
vocab_overlap = get_reader_vocab_overlap()

In [59]:
def vocab_overlap_crosstab(vocab_overlap):
    """
    Return an intuitively formatted `pd.DataFrame` giving vocab-overlap
    counts for all the datasets represented in `vocab_overlap`, the
    output of `get_reader_vocab_overlap`.
    """
    xtab = pd.crosstab(
        vocab_overlap['d1'],
        vocab_overlap['d2'],
        values=vocab_overlap['overlap'],
        aggfunc=np.mean)
    # Blank out the upper right to reduce visual clutter:
    for i in range(0, xtab.shape[0]):
        for j in range(i+1, xtab.shape[1]):
            xtab.iloc[i, j] = ''
    return xtab

In [60]:
vocab_overlap_crosstab(vocab_overlap)

d2,men,mturk771,simverb3500dev,simverb3500test,wordsim353
d1,,,,,
men,751,,,,
mturk771,230,1113,,,
simverb3500dev,23,67,536,,
simverb3500test,30,94,532,823,
wordsim353,86,158,13,17,437


This looks reasonable. By design, the SimVerb dev and test sets have a lot of overlap. The other overlap numbers are pretty small, even adjusting for dataset size.

### Pair overlap and score correlations

How many word pairs are shared across datasets and, for shared pairs, what is the correlation between their scores? That is, do the datasets agree?

In [61]:
def get_reader_pairs(reader):
    """
    Return the set of alphabetically-sorted word (str) tuples
    in `reader`
    """
    return {tuple(sorted([w1, w2])): score for w1, w2, score in reader()}

In [62]:
def get_reader_pair_overlap(readers=READERS):
    """Return a `pd.DataFrame` giving the number of overlapping
    word-pairs in pairs of readers, along with the Spearman
    correlations.
    """
    data = []
    for r1, r2 in itertools.product(READERS, repeat=2):
        if r1.__name__ != r2.__name__:
            d1 = get_reader_pairs(r1)
            d2 = get_reader_pairs(r2)
            overlap = []
            for p, s in d1.items():
                if p in d2:
                    overlap.append([s, d2[p]])
            if overlap:
                s1, s2 = zip(*overlap)
                rho = spearmanr(s1, s2)[0]
            else:
                rho = None
            # Canonical order for the pair:
            n1, n2 = sorted([get_reader_name(r1), get_reader_name(r2)])
            d = {
                'd1': n1,
                'd2': n2,
                'pair_overlap': len(overlap),
                'rho': rho}
            data.append(d)
    df = pd.DataFrame(data)
    df = df.sort_values(['pair_overlap','d1','d2'], ascending=False)
    # Return only every other row to avoid repeats:
    return df[::2].reset_index(drop=True)

In [63]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    display(get_reader_pair_overlap())

,d1,d2,pair_overlap,rho
0,men,mturk771,11,0.592191
1,men,wordsim353,5,0.700000
2,mturk771,simverb3500test,4,0.400000
3,men,simverb3500test,2,1.000000
4,simverb3500dev,simverb3500test,1,NaN
5,simverb3500test,wordsim353,0,NaN
6,simverb3500dev,wordsim353,0,NaN
7,mturk771,wordsim353,0,NaN
8,mturk771,simverb3500dev,0,NaN
9,men,simverb3500dev,0,NaN


This looks reasonable: none of the datasets have a lot of overlapping pairs, so we don't have to worry too much about places where they give conflicting scores.

## Creating Model

NameError: name 'd1' is not defined

## Evaluation

This section builds up the evaluation code that you'll use for the homework and bake-off. For illustrations, I'll read in a VSM created from `data/vsmdata/giga_window5-scaled.csv.gz`:

In [335]:
giga5 = pd.read_csv(
    os.path.join(VSM_HOME, "giga_window5-scaled.csv.gz"), index_col=0)

### Dataset evaluation

In [336]:
def word_similarity_evaluation(reader, df, distfunc=vsm.cosine):
    """
    Word-similarity evalution framework.

    Parameters
    ----------
    reader : iterator
        A reader for a word-similarity dataset. Just has to yield
        tuples (word1, word2, score).

    df : pd.DataFrame
        The VSM being evaluated.

    distfunc : function mapping vector pairs to floats.
        The measure of distance between vectors. Can also be
        `vsm.euclidean`, `vsm.matching`, `vsm.jaccard`, as well as
        any other float-valued function on pairs of vectors.

    Raises
    ------
    ValueError
        If `df.index` is not a subset of the words in `reader`.

    Returns
    -------
    float, data
        `float` is the Spearman rank correlation coefficient between
        the dataset scores and the similarity values obtained from
        `df` using  `distfunc`. This evaluation is sensitive only to
        rankings, not to absolute values.  `data` is a `pd.DataFrame`
        with columns['word1', 'word2', 'score', 'distance'].

    """
    data = []
    for w1, w2, score in reader():
        d = {'word1': w1, 'word2': w2, 'score': score}
        for w in [w1, w2]:
            if w not in df.index:
                raise ValueError(
                    "Word '{}' is in the similarity dataset {} but not in the "
                    "DataFrame, making this evaluation ill-defined. Please "
                    "switch to a DataFrame with an appropriate vocabulary.".
                    format(w, get_reader_name(reader)))
        d['distance'] = distfunc(df.loc[w1], df.loc[w2])
        data.append(d)
    data = pd.DataFrame(data)
    rho, pvalue = spearmanr(data['score'].values, data['distance'].values)
    return rho, data

In [337]:
rho, eval_df = word_similarity_evaluation(men_reader, giga5)

In [338]:
rho

0.40375964105441753

In [339]:
eval_df.head()

,word1,word2,score,distance
0,sun,sunlight,-50.0,0.956828
1,automobile,car,-50.0,0.979143
2,river,water,-49.0,0.970105
3,stairs,staircase,-49.0,0.980475
4,morning,sunrise,-49.0,0.963624


### Dataset error analysis

For error analysis, we can look at the words with the largest delta between the gold score and the distance value in our VSM. We do these comparisons based on ranks, just as with our primary metric (Spearman $\rho$), and we normalize both rankings so that they have a comparable number of levels.

In [340]:
def word_similarity_error_analysis(eval_df):
    eval_df['distance_rank'] = _normalized_ranking(eval_df['distance'])
    eval_df['score_rank'] = _normalized_ranking(eval_df['score'])
    eval_df['error'] =  abs(eval_df['distance_rank'] - eval_df['score_rank'])
    return eval_df.sort_values('error')


def _normalized_ranking(series):
    ranks = series.rank(method='dense')
    return ranks / ranks.sum()

Best predictions:

In [341]:
word_similarity_error_analysis(eval_df).head()

,word1,word2,score,distance,distance_rank,score_rank,error
1041,hummingbird,pelican,-32.0,0.975007,0.000243,0.000244,2.434543e-07
2315,lily,pigs,-13.0,0.980834,0.000488,0.000487,4.016842e-07
2951,bucket,girls,-4.0,0.983473,0.000602,0.000603,4.151568e-07
150,night,sunset,-43.0,0.968690,0.000102,0.000103,6.520315e-07
2062,oak,petals,-17.0,0.979721,0.000435,0.000436,7.162632e-07


Worst predictions:

In [342]:
word_similarity_error_analysis(eval_df).tail()

,word1,word2,score,distance,distance_rank,score_rank,error
67,branch,twigs,-45.0,0.984622,0.000630,0.000077,0.000553
190,birds,stork,-43.0,0.987704,0.000657,0.000103,0.000554
185,bloom,tulip,-43.0,0.990993,0.000663,0.000103,0.000561
167,bloom,blossom,-43.0,0.991760,0.000664,0.000103,0.000561
198,bloom,rose,-43.0,0.992406,0.000664,0.000103,0.000561


### Full evaluation

A full evaluation is just a loop over all the readers on which one want to evaluate, with a macro-average at the end:

In [343]:
def full_word_similarity_evaluation(df, readers=READERS, distfunc=vsm.cosine):
    """
    Evaluate a VSM against all datasets in `readers`.

    Parameters
    ----------
    df : pd.DataFrame

    readers : tuple
        The similarity dataset readers on which to evaluate.

    distfunc : function mapping vector pairs to floats.
        The measure of distance between vectors. Can also be
        `vsm.euclidean`, `vsm.matching`, `vsm.jaccard`, as well as
        any other float-valued function on pairs of vectors.

    Returns
    -------
    pd.Series
        Mapping dataset names to Spearman r values.

    """
    scores = {}
    for reader in readers:
        score, data_df = word_similarity_evaluation(reader, df, distfunc=distfunc)
        scores[get_reader_name(reader)] = score
    series = pd.Series(scores, name='Spearman r')
    series['Macro-average'] = series.mean()
    return series

In [344]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    display(full_word_similarity_evaluation(giga5))

NameError: name 'get_reader_name' is not defined

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### PPMI as a baseline [0.5 points]

The insight behind PPMI is a recurring theme in word representation learning, so it is a natural baseline for our task. For this question, write a function called `run_giga_ppmi_baseline` that does the following:

1. Reads the Gigaword count matrix with a window of 20 and a flat scaling function into a `pd.DataFrame`s, as is done in the VSM notebooks. The file is `data/vsmdata/giga_window20-flat.csv.gz`, and the VSM notebooks provide examples of the needed code.

1. Reweights this count matrix with PPMI.

1. Evaluates this reweighted matrix using `full_word_similarity_evaluation`. The return value of `run_giga_ppmi_baseline` should be the return value of this call to `full_word_similarity_evaluation`.

The goal of this question is to help you get more familiar with the code in `vsm` and the function `full_word_similarity_evaluation`.

The function `test_run_giga_ppmi_baseline` can be used to test that you've implemented this specification correctly.

In [345]:
def run_giga_ppmi_baseline():
    pass
    ##### YOUR CODE HERE



In [346]:
def test_run_giga_ppmi_baseline(func):
    """`func` should be `run_giga_ppmi_baseline"""
    result = func()
    ws_result = result.loc['wordsim353'].round(2)
    ws_expected = 0.58
    assert ws_result == ws_expected, \
        "Expected wordsim353 value of {}; got {}".format(
            ws_expected, ws_result)

In [347]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_giga_ppmi_baseline(run_giga_ppmi_baseline)

AttributeError: 'NoneType' object has no attribute 'loc'

### Gigaword with LSA at different dimensions [0.5 points]

We might expect PPMI and LSA to form a solid pipeline that combines the strengths of PPMI with those of dimensionality reduction. However, LSA has a hyper-parameter $k$ – the dimensionality of the final representations – that will impact performance. For this problem, write a wrapper function `run_ppmi_lsa_pipeline` that does the following:

1. Takes as input a count `pd.DataFrame` and an LSA parameter `k`.
1. Reweights the count matrix with PPMI.
1. Applies LSA with dimensionality `k`.
1. Evaluates this reweighted matrix using `full_word_similarity_evaluation`. The return value of `run_ppmi_lsa_pipeline` should be the return value of this call to `full_word_similarity_evaluation`.

The goal of this question is to help you get a feel for how much LSA alone can contribute to this problem. 

The  function `test_run_ppmi_lsa_pipeline` will test your function on the count matrix in `data/vsmdata/giga_window20-flat.csv.gz`.

In [348]:
def run_ppmi_lsa_pipeline(count_df, k):
    pass
    ##### YOUR CODE HERE



In [349]:
def test_run_ppmi_lsa_pipeline(func):
    """`func` should be `run_ppmi_lsa_pipeline`"""
    giga20 = pd.read_csv(
        os.path.join(VSM_HOME, "giga_window20-flat.csv.gz"), index_col=0)
    results = func(giga20, k=10)
    men_expected = 0.57
    men_result = results.loc['men'].round(2)
    assert men_result == men_expected,\
        "Expected men value of {}; got {}".format(men_expected, men_result)

In [350]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_ppmi_lsa_pipeline(run_ppmi_lsa_pipeline)

AttributeError: 'NoneType' object has no attribute 'loc'

### Gigaword with GloVe [0.5 points]

Can GloVe improve over the PPMI-based baselines we explored above? To begin to address this question, let's run GloVe and see how performance on our task changes throughout the optimization process.

__Your task__: write a function `run_glove_wordsim_evals` that does the following:

1. Has a parameter `n_runs` with default value `5`.

1. Reads in `data/vsmdata/giga_window5-scaled.csv.gz`.

1. Creates a `TorchGloVe` instance with `warm_start=True`, `max_iter=50`, and all other parameters set to their defaults.

1. `n_runs` times, calls `fit` on your model and, after each, runs `full_word_similarity_evaluation` with default keyword parameters, extract the 'Macro-average' score, and add that score to a list.

1. Returns the list of scores created.

The trend should give you a sense for whether it is worth running GloVe for more iterations.

Some implementation notes:

* `TorchGloVe` will accept and return `pd.DataFrame` instances, so you shouldn't need to do any type conversions.

* Performance will vary a lot for this function, so there is some uncertainty in the testing, but `run_glove_wordsim_evals` will at least check that you wrote a function with the right general logic.

In [ ]:
def run_glove_wordsim_evals(n_runs=5):

    from torch_glove import TorchGloVe

    ##### YOUR CODE HERE



In [ ]:
def test_run_small_glove_evals(data):
    """`data` should be the return value of `run_glove_wordsim_evals`"""
    assert isinstance(data, list), \
        "`run_glove_wordsim_evals` should return a list"
    assert all(isinstance(x, float) for x in data), \
        ("All the values in the list returned by `run_glove_wordsim_evals` "
         "should be floats.")

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    glove_scores = run_glove_wordsim_evals()
    print(glove_scores)
    test_run_small_glove_evals(glove_scores)

### Dice coefficient [0.5 points]

Implement the Dice coefficient for real-valued vectors, as

$$
\textbf{dice}(u, v) = 
1 - \frac{
  2 \sum_{i=1}^{n}\min(u_{i}, v_{i})
}{
    \sum_{i=1}^{n} u_{i} + v_{i}
}$$
 
You can use `test_dice_implementation` below to check that your implementation is correct.

In [353]:
def dice(u, v):
    pass
    ##### YOUR CODE HERE



In [354]:
def test_dice_implementation(func):
    """`func` should be an implementation of `dice` as defined above."""
    X = np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   8.,  10.,   0.],
        [  0.,  18.,   0.,   5.]])
    assert func(X[0], X[1]).round(5) == 0.80198
    assert func(X[1], X[2]).round(5) == 0.67568

In [355]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_dice_implementation(dice)

AttributeError: 'NoneType' object has no attribute 'round'

### t-test reweighting [2 points]



The t-test statistic can be thought of as a reweighting scheme. For a count matrix $X$, row index $i$, and column index $j$:

$$\textbf{ttest}(X, i, j) = 
\frac{
    P(X, i, j) - \big(P(X, i, *)P(X, *, j)\big)
}{
\sqrt{(P(X, i, *)P(X, *, j))}
}$$

where $P(X, i, j)$ is $X_{ij}$ divided by the total values in $X$, $P(X, i, *)$ is the sum of the values in row $i$ of $X$ divided by the total values in $X$, and $P(X, *, j)$ is the sum of the values in column $j$ of $X$ divided by the total values in $X$.

For this problem, implement this reweighting scheme. You can use `test_ttest_implementation` below to check that your implementation is correct. You do not need to use this for any evaluations, though we hope you will be curious enough to do so!

In [356]:
def ttest(df):
    pass
    ##### YOUR CODE HERE



In [357]:
def test_ttest_implementation(func):
    """`func` should be `ttest`"""
    X = pd.DataFrame(np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   8.,  10.,   0.],
        [  0.,  18.,   0.,   5.]]))
    actual = np.array([
        [ 0.33056, -0.07689,  0.04321, -0.10532],
        [-0.07689,  0.03839, -0.10874,  0.07574],
        [ 0.04321, -0.10874,  0.36111, -0.14894],
        [-0.10532,  0.07574, -0.14894,  0.05767]])
    predicted = func(X)
    assert np.array_equal(predicted.round(5), actual)

In [358]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_ttest_implementation(ttest)

AttributeError: 'NoneType' object has no attribute 'round'

### Enriching a VSM with subword information [2 points]

It might be useful to combine character-level information with word-level information. To help you begin asssessing this idea, this question asks you to write a function that modifies an existing VSM so that the representation for each word $w$ is the element-wise sum of $w$'s original word-level representation with all the representations for the n-grams $w$ contains. 

The following starter code should help you structure this and clarify the requirements, and a simple test is included below as well.

You don't need to write a lot of code; the motivation for this question is that the function you write could have practical value.

In [ ]:
def subword_enrichment(df, n=4):
    pass
    # 1. Use `vsm.ngram_vsm` to create a character-level
    # VSM from `df`, using the above parameter `n` to
    # set the size of the ngrams.

    ##### YOUR CODE HERE


    # 2. Use `vsm.character_level_rep` to get the representation
    # for every word in `df` according to the character-level
    # VSM you created above.

    ##### YOUR CODE HERE


    # 3. For each representation created at step 2, add in its
    # original representation from `df`. (This should use
    # element-wise addition; the dimensionality of the vectors
    # will be unchanged.)

    ##### YOUR CODE HERE


    # 4. Return a `pd.DataFrame` with the same index and column
    # values as `df`, but filled with the new representations
    # created at step 3.

    ##### YOUR CODE HERE



In [ ]:
def test_subword_enrichment(func):
    """`func` should be an implementation of subword_enrichment as
    defined above.
    """
    vocab = ["ABCD", "BCDA", "CDAB", "DABC"]
    df = pd.DataFrame([
        [1, 1, 2, 1],
        [3, 4, 2, 4],
        [0, 0, 1, 0],
        [1, 0, 0, 0]], index=vocab)
    expected = pd.DataFrame([
        [14, 14, 18, 14],
        [22, 26, 18, 26],
        [10, 10, 14, 10],
        [14, 10, 10, 10]], index=vocab)
    new_df = func(df, n=2)
    assert np.array_equal(expected.columns, new_df.columns), \
        "Columns are not the same"
    assert np.array_equal(expected.index, new_df.index), \
        "Indices are not the same"
    assert np.array_equal(expected.values, new_df.values), \
        "Co-occurrence values aren't the same"

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_subword_enrichment(subword_enrichment)

### Your original system [3 points]

This question asks you to design your own model. You can of course include steps made above (ideally, the above questions informed your system design!), but your model should not be literally identical to any of the above models. Other ideas: retrofitting, autoencoders, GloVe, subword modeling, ... 

Requirements:

1. Your code must operate on one or more of the count matrices in `data/vsmdata`. You can choose which subset of them; this is an important design feature of your system. __Other pretrained vectors cannot be introduced__.

1. Retrofitting is permitted.

1. Your code must be self-contained, so that we can work with your model directly in your homework submission notebook. If your model depends on external data or other resources, please submit a ZIP archive containing these resources along with your submission.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM 
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# STOP COMMENT: Please do not remove this comment.

## Bake-off [1 point]

For the bake-off, we will release two additional datasets. The announcement will go out on the discussion forum. We will also release reader code for these datasets that you can paste into this notebook. You will evaluate your custom model $M$ (from the previous question) on these new datasets using `full_word_similarity_evaluation`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
# Enter your bake-off assessment code into this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE


In [ ]:
# On an otherwise blank line in this cell, please enter
# your "Macro-average" value as reported by the code above.
# Please enter only a number between 0 and 1 inclusive.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
